<h2>Importing all necessary libraries</h2>

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt 
import plotly.express as px
from sklearn.model_selection import train_test_split 

<h2>Reading the data</h2>

In [ ]:
rain_df = pd.read_csv('data/weatherAUS.csv')

<h2>Exploring Data</h2>

In [ ]:
rain_df.dropna(subset=['RainToday','RainTomorrow'], inplace=True)

In [ ]:
rain_df['RainToday'] = rain_df.RainToday.map({'No':0, 'Yes':1}).astype(int)
rain_df['RainTomorrow'] = rain_df.RainTomorrow.map({'No':0, 'Yes':1}).astype(int)

In [ ]:
numerical_cols = rain_df.select_dtypes(include=['int32','float64']).columns.tolist()
cat_cols = rain_df.select_dtypes('object').columns.tolist()

<h2>Rainfall</h2>

In [ ]:
px.histogram(rain_df, x='Rainfall', color='RainTomorrow')

In [ ]:
rain_df['Rainfall'].skew()

In [ ]:
rain_df['Rainfall'] = np.log1p(rain_df['Rainfall'])

In [ ]:
px.histogram(rain_df.sample(2000),x='Rainfall', color='RainTomorrow')

<h2>Gaussian Imputer</h2>

In [ ]:
def GaussianImpute(col):
    mean = col.mean()
    var = col.std()

    is_na = col.isna()
    no_of_missing_values = is_na.sum()
    
    imputed_values = np.random.normal(mean, var, size=no_of_missing_values)
    
    imputed_col = col.copy()
    imputed_col[is_na] = imputed_values
    
    return imputed_col

<h2>Primary Idea Function</h2>

In [ ]:
def display(col):
    print('Skew:', rain_df[col].skew())
    print('Cor:',rain_df[col].corr(rain_df.RainTomorrow))
    print('No of missing values:', rain_df[col].isna().sum())
    fig = px.histogram(rain_df, x=col, color='RainTomorrow')
    fig.show()

<h2>WindGusSpeed</h2>

In [ ]:
display('WindGustSpeed')

In [ ]:
rain_df['WindGustSpeed'] = GaussianImpute(rain_df['WindGustSpeed'])

In [ ]:
display('WindGustSpeed')

In [ ]:
rain_df['WindGustSpeed'].describe()

In [ ]:
rain_df['WindGustSpeed'] = np.log1p(rain_df['WindGustSpeed'])

<h2>Humidity</h2>

In [ ]:
display('Humidity3pm')

In [ ]:
sns.histplot(rain_df['Humidity3pm'], kde=True)

In [ ]:
rain_df['Humidity3pm'].describe()

In [ ]:
rain_df['Humidity3pm'] = GaussianImpute(rain_df['Humidity3pm'])

In [ ]:
display('Humidity3pm')

<h2>Pressure</h2>

In [ ]:
display('Pressure9am')

In [ ]:
rain_df['Pressure9am'] = GaussianImpute(rain_df['Pressure9am'])

In [ ]:
display('Pressure9am')

<h2>Temp</h2>

In [ ]:
display('Temp3pm')

In [ ]:
rain_df['Temp3pm'] = GaussianImpute(rain_df['Temp3pm'])

In [ ]:
display('Temp3pm')

<h2>Creating new column: Rain Risk</h2>

In [ ]:
rain_df['rain_risk'] = (rain_df['Humidity3pm']/100) * (1-(rain_df['Temp3pm'])/100) * rain_df['WindGustSpeed']

In [ ]:
display('rain_risk')

<h2>Creating new column: Temp Diff</h2>

In [ ]:
rain_df['MaxTemp'] = GaussianImpute(rain_df['MaxTemp'])
rain_df['MinTemp'] = GaussianImpute(rain_df['MinTemp'])

In [ ]:
rain_df['temp_dif'] = rain_df['MaxTemp']-rain_df['MinTemp']

In [ ]:
display('temp_dif')

<h2>Defining Columns</h2>

In [ ]:
numeric_cols = ['Rainfall','WindGustSpeed','Humidity3pm','Pressure9am','Temp3pm', 'RainToday']
numeric_cols

In [ ]:
cat_cols = ['Location', 'WindDir3pm']
cat_cols

In [ ]:
target_cols = 'RainTomorrow'

In [ ]:
input_cols = numeric_cols + cat_cols
input_cols

In [ ]:
rain_df.columns

<h2>Imputing Categorical Columns</h2>

In [ ]:
rain_df['WindDir3pm'] = rain_df['WindDir3pm'].fillna(rain_df['WindDir3pm'].mode()[0])

In [ ]:
rain_df[cat_cols].isna().sum()

In [ ]:
px.histogram(rain_df, x='WindDir3pm', color='RainTomorrow')

In [ ]:
rain_df[cat_cols].describe()

<h2>Spliting Train, Validation, Test Data</h2>

In [ ]:
sns.countplot(x=pd.to_datetime(rain_df.Date).dt.year)

In [ ]:
train_df = rain_df[year<2015]
val_df = rain_df[year == 2015]
test_df = rain_df[year>2015]

In [ ]:
train_inputs = train_df[input_cols].copy()
train_target = train_df[target_col].copy()
val_inputs = val_df[input_cols].copy()
val_target = val_df[target_col].copy()
test_inputs = test_df[input_cols].copy()
test_target = test_df[target_col].copy()

<h1>Scaling Down</h1>
<p>In this section, I am trying to scale down all the numeric columns values to create a level playing field using MinMaxScaler</p>
<p>This will bar the model to be more influenced by the larger value columns</p>

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(train_df[numeric_cols])

In [ ]:
train_inputs[numeric_cols] = scaler.transform(train_inputs[numeric_cols])
val_inputs[numeric_cols] = scaler.transform(val_inputs[numeric_cols])
test_inputs[numeric_cols] = scaler.transform(test_inputs[numeric_cols])

In [ ]:
train_inputs[numeric_cols].skew()

In [ ]:
import os
os.getcwd()

<h1>Encoder for Categorical Columns</h1>
<p>Since I dealt with numeric columns with MinMaxScaler, now I am using OneHotEncoder to cope with categorical columns</p>

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

In [ ]:
train_inputs[cat_cols].isna().sum()

In [ ]:
encoder.fit(train_inputs[cat_cols])

In [ ]:
encoded_cols = encoder.get_feature_names_out(cat_cols)
encoded_cols

In [ ]:
train_inputs[encoded_cols] = encoder.transform(train_inputs[cat_cols])
val_inputs[encoded_cols] = encoder.transform(val_inputs[cat_cols])
test_inputs[encoded_cols] = encoder.transform(test_inputs[cat_cols])

<h2>Logistic Regression Model</h2>

In [ ]:
from sklearn.linear_model import LogisticRegression 

In [ ]:
model = LogisticRegression(solver='liblinear',max_iter=100)

In [ ]:
model.fit(train_inputs[numeric_cols + list(encoded_cols)], train_target)

In [ ]:
print(model.coef_.tolist())

In [ ]:
weight_df = pd.DataFrame({
    'feature': numeric_cols + encoded_cols.tolist(),
    'weight': model.coef_.tolist()[0]
})
weight_df.sort_values(by='weight', ascending=False)

In [ ]:
model.intercept_

In [ ]:
sns.barplot(data=weight_df.sort_values(by='weight', ascending=False).head(10), x='weight', y='feature')

In [ ]:
px.histogram(rain_df, x='Cloud3pm', color='RainTomorrow')

In [ ]:
x_train = train_inputs[numeric_cols + encoded_cols.tolist()]
x_val = val_inputs[numeric_cols + encoded_cols.tolist()]
x_test = test_inputs[numeric_cols + encoded_cols.tolist()]

In [ ]:
train_predict = model.predict(x_train)

In [ ]:
train_predict

In [ ]:
train_target

In [ ]:
np.sum(np.array(train_predict) != np.array(train_target))

In [ ]:
train_target.shape

<h2>Accuracy and Confusion Matrix</h2>

In [ ]:
from sklearn.metrics import accuracy_score 

In [ ]:
accuracy_score(train_target, train_predict)

In [ ]:
train_probabs = model.predict_proba(x_train)
train_probabs

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(train_target, train_predict, normalize='true')

In [ ]:
def conf_mat(inputs, target, name=''):
    predict = model.predict(inputs)
    acc_score = accuracy_score(target, predict)
    print("Accuracy: {:.2f}%".format(acc_score*100))
    matrix = confusion_matrix(target, predict, normalize='true')

    plt.figure()
    sns.heatmap(matrix, annot=True)
    plt.xlabel('Prediction')
    plt.ylabel('Target')
    plt.title('{} Confusion Matrix'.format(name))

    return predict

In [ ]:
train_predict = conf_mat(x_train, train_target, 'Training')

In [ ]:
val_predict = conf_mat(x_val, val_target, 'Validation')

In [ ]:
test_predict = conf_mat(x_test, test_target, 'Test')